<a href="https://colab.research.google.com/github/swago72/cx-knowledge-assistant/blob/main/cx_knowledge_assistant_week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-text-splitters -q

In [2]:
!pip install langchain chromadb sentence-transformers beautifulsoup4 lxml -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
REPO_PATH = "/content/drive/MyDrive/cx-knowledge-assistant"
os.chdir(REPO_PATH)

!pwd
!ls
!git status


Mounted at /content/drive
/content/drive/MyDrive/cx-knowledge-assistant
'Cancel, pause, or change a subscription on Google Play - Computer - Google Play Help.html'
'Check the status of a refund request for Google Play - Google Play Help.html'
 chroma_db
 cx_knowledge_assistant_week1.ipynb
 data
'Fix payment issues on your account - Computer - Google Play Help.html'
'Fix problems downloading apps with basic troubleshooting steps - Google Play Help.html'
'Fix problems downloading one app - Google Play Help.html'
'Fix problems with subscriptions - Computer - Google Play Help.html'
'Fix problems with the Google Play Store app - Google Play Help.html'
'Learn about Google Play refund policies - Google Play Help.html'
 notebooks
'Problems with in-app purchases - Google Play Help.html'
 README.md
"Report charges you don't recognize - Google Play Help.html"
'Request a refund on Google Play - Google Play Help.html'
 requirements.txt
 src
'Subscribe to services or content - Computer - Google Play

In [4]:
!git config --global user.name "Rakshit Mathur"
!git config --global user.email "rakshit.mathur38@gmail.com"
!git config --global --list | grep user


user.name=Rakshit Mathur
user.email=rakshit.mathur38@gmail.com


In [5]:
import os
for folder in ["src", "data/html", "chroma_db", "notebooks"]:
    os.makedirs(folder, exist_ok=True)

print("✓ Folder structure ready")
!find . -maxdepth 2 -type d


✓ Folder structure ready
.
./.git
./.git/hooks
./.git/info
./.git/branches
./.git/refs
./.git/objects
./.git/logs
./src
./data
./data/html
./chroma_db
./chroma_db/f27d2b5d-40b9-468c-83c0-a864bd453561
./notebooks


In [6]:
from google.colab import files

print("Select ALL your HTML files now")
uploaded = files.upload()

import os
os.makedirs("data/html", exist_ok=True)

for filename, content in uploaded.items():
    with open(f"data/html/{filename}", "wb") as f:
        f.write(content)

print(f"✓ Saved {len(uploaded)} HTML files into data/html/")
!ls data/html | head


Select ALL your HTML files now


Saving Subscribe with Google - Google Play Help.html to Subscribe with Google - Google Play Help (1).html
Saving Subscribe to services or content - Computer - Google Play Help.html to Subscribe to services or content - Computer - Google Play Help (1).html
Saving Request a refund on Google Play - Google Play Help.html to Request a refund on Google Play - Google Play Help (1).html
Saving Report charges you don't recognize - Google Play Help.html to Report charges you don't recognize - Google Play Help (1).html
Saving Problems with in-app purchases - Google Play Help.html to Problems with in-app purchases - Google Play Help (1).html
Saving Learn about Google Play refund policies - Google Play Help.html to Learn about Google Play refund policies - Google Play Help (1).html
Saving Fix problems with the Google Play Store app - Google Play Help.html to Fix problems with the Google Play Store app - Google Play Help (1).html
Saving Fix problems with subscriptions - Computer - Google Play Help.h

In [7]:
import glob, shutil, os

os.makedirs("data/html", exist_ok=True)

root_html = glob.glob("*.html")
for f in root_html:
    shutil.move(f, os.path.join("data/html", f))

print(f"✓ Moved {len(root_html)} root HTML files into data/html/")


✓ Moved 26 root HTML files into data/html/


In [8]:
ingestion_code = r'''
from bs4 import BeautifulSoup
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb
import os
import re

def _safe_chunk_id(source: str, i: int) -> str:
    base = re.sub(r"[^a-zA-Z0-9_]+", "_", source)
    return f"{base[:80]}_chunk_{i}"

def load_html_files(folder_path="data/html"):
    documents = []
    for filename in os.listdir(folder_path):
        if not filename.lower().endswith(".html"):
            continue

        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            raw_html = f.read()

        soup = BeautifulSoup(raw_html, "lxml")
        for tag in soup(["script", "style", "nav", "header", "footer", "aside", "noscript", "iframe"]):
            tag.decompose()

        clean_text = soup.get_text(separator="\\n", strip=True)
        lines = [l for l in clean_text.splitlines() if l.strip()]
        clean_text = "\\n".join(lines)

        if len(clean_text) < 100:
            print(f"  ⚠ Skipping {filename} — too short")
            continue

        short_name = filename.replace(" - Google Play Help.html", "")

        documents.append({
            "content": clean_text,
            "source": filename,
            "short_name": short_name,
            "chars": len(clean_text)
        })
        print(f"  ✓ {short_name} ({len(clean_text):,} chars)")

    print(f"\\nTotal documents loaded: {len(documents)}")
    return documents

def chunk_documents(documents):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separators=["\\n\\n", "\\n", ".", " "]
    )

    chunks = []
    for doc in documents:
        texts = splitter.split_text(doc["content"])
        for i, text in enumerate(texts):
            chunks.append({
                "text": text,
                "source": doc["source"],
                "short_name": doc["short_name"],
                "chunk_id": _safe_chunk_id(doc["source"], i),
                "chunk_num": i,
                "total": len(texts),
            })

    print(f"Total chunks created: {len(chunks)}")
    return chunks

def embed_and_store(chunks, collection_name="cx_knowledge_base"):
    print("Loading HuggingFace model (downloads once)...")
    model = SentenceTransformer("all-MiniLM-L6-v2")
    print("✓ Model ready\\n")

    client = chromadb.PersistentClient(path="./chroma_db")
    collection = client.get_or_create_collection(
        name=collection_name,
        metadata={"hnsw:space": "cosine"}
    )

    existing = collection.get(include=[])
    existing_ids = set(existing["ids"]) if existing and "ids" in existing else set()

    new_chunks = [c for c in chunks if c["chunk_id"] not in existing_ids]
    if not new_chunks:
        print("Already embedded. Nothing to add.")
        print(f"✓ ChromaDB contains {collection.count()} total chunks")
        return collection, model

    print(f"Embedding {len(new_chunks)} chunks...")
    batch_size = 50

    for i in range(0, len(new_chunks), batch_size):
        batch = new_chunks[i:i + batch_size]
        texts = [c["text"] for c in batch]
        embeds = model.encode(texts, show_progress_bar=False).tolist()

        collection.add(
            embeddings=embeds,
            documents=[c["text"] for c in batch],
            metadatas=[{
                "source": c["source"],
                "short_name": c["short_name"],
                "chunk_num": c["chunk_num"]
            } for c in batch],
            ids=[c["chunk_id"] for c in batch]
        )

        done = min(i + batch_size, len(new_chunks))
        print(f"  Stored {done}/{len(new_chunks)}")

    print(f"\\n✓ ChromaDB contains {collection.count()} total chunks")
    return collection, model

def main():
    print("=" * 55)
    print("  CX Knowledge Assistant — Ingestion Pipeline")
    print("=" * 55 + "\\n")

    print("STEP 1: Loading HTML files...")
    documents = load_html_files()

    print("\\nSTEP 2: Chunking...")
    chunks = chunk_documents(documents)

    print("\\nSTEP 3: Embedding and storing in ChromaDB...")
    embed_and_store(chunks)

    print("\\n✅ Done. Knowledge base is ready for Week 2.")

if __name__ == "__main__":
    main()
'''

with open("src/ingestion.py", "w", encoding="utf-8") as f:
    f.write(ingestion_code)

print("✓ Created src/ingestion.py")
!ls src


✓ Created src/ingestion.py
ingestion.py


In [9]:
requirements = """langchain
langchain-text-splitters
langchain-google-genai
chromadb
sentence-transformers
beautifulsoup4
lxml
streamlit
pandas
python-dotenv
"""

gitignore = """.env
chroma_db/
data/html/
__pycache__/
*.pyc
.ipynb_checkpoints/
"""

readme = r"""# CX Knowledge Assistant

An AI-powered RAG chatbot that lets support agents query documentation using plain English and get cited answers.

## Stack
- LLM: Google Gemini (via LangChain)
- Embeddings: HuggingFace all-MiniLM-L6-v2
- Vector DB: ChromaDB
- Orchestration: LangChain
- UI: Streamlit

## Setup
    pip install -r requirements.txt
    # Add GOOGLE_API_KEY to .env
    python src/ingestion.py
    streamlit run app.py

## Status
- [x] Week 1: Ingestion pipeline
- [ ] Week 2: RAG + Gemini
- [ ] Week 3: Analytics
- [ ] Week 4: Streamlit UI
- [ ] Week 5: Polish
- [ ] Week 6: Deploy
"""

with open("requirements.txt", "w", encoding="utf-8") as f:
    f.write(requirements)

with open(".gitignore", "w", encoding="utf-8") as f:
    f.write(gitignore)

with open("README.md", "w", encoding="utf-8") as f:
    f.write(readme)

print("✓ Wrote requirements.txt, .gitignore, README.md")
!ls


✓ Wrote requirements.txt, .gitignore, README.md
chroma_db			    data       README.md	 src
cx_knowledge_assistant_week1.ipynb  notebooks  requirements.txt


In [10]:
!pip -q install -r requirements.txt
!python src/ingestion.py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.1 MB/s eta 0:00:00
  CX Knowledge Assistant — Ingestion Pipeline
=======================================================\n
STEP 1: Loading HTML files...
  ✓ Subscribe with Google (2,231 chars)
  ✓ Subscribe to services or content - Computer (11,479 chars)
  ✓ Request a refund on Google Play (2,828 chars)
  ✓ Report charges you don't recognize (4,010 chars)
  ✓ Problems with in-app purchases (5,483 chars)
  ✓ Learn about Google Play refund policies (2,832 chars)
  ✓ Fix problems with the Google Play Store app (3,851 chars)
  ✓ Fix problems with subscriptions - Computer (5,771 chars)
  ✓ Fix problems downloading one app (2,353 chars)
  ✓ Fix problems downloading apps with basic troubleshooting steps (3,538 chars)
  ✓ Fix payment issues on your account - Computer (8,775 chars)
 

In [11]:
!git status
!git add src/ingestion.py requirements.txt .gitignore README.md
!git commit -m "feat: Week 1 ingestion pipeline (BeautifulSoup + HF embeddings + ChromaDB)"
!git push origin main


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   README.md
	modified:   src/ingestion.py

no changes added to commit (use "git add" and/or "git commit -a")
[main fdfe3a8] feat: Week 1 ingestion pipeline (BeautifulSoup + HF embeddings + ChromaDB)
 2 files changed, 18 insertions(+), 20 deletions(-)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 611 bytes | 67.00 KiB/s, done.
Total 5 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/swago72/cx-knowledge-assistant.git
   6163e3b..fdfe3a8  main -> main


In [12]:
!git remote set-url origin https://github.com/swago72/cx-knowledge-assistant.git
!git remote -v


origin	https://github.com/swago72/cx-knowledge-assistant.git (fetch)
origin	https://github.com/swago72/cx-knowledge-assistant.git (push)
